In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Chargement des données
df = pd.read_csv('../data/df_venues_processed.csv', sep=';')

# Conversion de la colonne Date en datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Affichage des premières lignes
print("Premières lignes du dataset:")
print(df.head())
print(f"\nNombre de lignes: {len(df)}")
print(f"\nColonnes: {df.columns.tolist()}")

In [ ]:
# Visualisation 1: Évolution du taux de présence global au fil du temps
plt.figure(figsize=(15, 6))
plt.plot(df['Date'], df['GLOBAL'], linewidth=1.5, color='#2E86AB')
plt.title('Évolution du taux de présence (GLOBAL)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Taux de présence', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Visualisation 2: Comparaison du taux de présence par catégorie (D1-D4)
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

categories = ['D1', 'D2', 'D3', 'D4']
colors = ['#A23B72', '#F18F01', '#C73E1D', '#6A994E']

for idx, (cat, color) in enumerate(zip(categories, colors)):
    ax = axes[idx // 2, idx % 2]
    ax.plot(df['Date'], df[cat], linewidth=1.5, color=color)
    ax.set_title(f'Taux de présence {cat}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Date', fontsize=10)
    ax.set_ylabel('Taux de présence', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Visualisation 3: Distribution du taux de présence par jour de la semaine
presence_par_jour = df.groupby('jour_semaine')['GLOBAL'].mean().reindex(
    ['lundi', 'mardi', 'mercredi', 'jeudi', 'vendredi']
)

plt.figure(figsize=(12, 6))
bars = plt.bar(presence_par_jour.index, presence_par_jour.values, 
               color=['#264653', '#2A9D8F', '#E9C46A', '#F4A261', '#E76F51'])
plt.title('Taux de présence moyen par jour de la semaine', fontsize=14, fontweight='bold')
plt.xlabel('Jour de la semaine', fontsize=12)
plt.ylabel('Taux de présence moyen', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')

# Ajout des valeurs au-dessus des barres
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.0f}',
             ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Visualisation 4: Impact de la température sur le taux de présence
plt.figure(figsize=(12, 6))
plt.scatter(df['Temp'], df['GLOBAL'], alpha=0.5, s=30, color='#E63946')
plt.title('Relation entre température et taux de présence', fontsize=14, fontweight='bold')
plt.xlabel('Température (°C)', fontsize=12)
plt.ylabel('Taux de présence (GLOBAL)', fontsize=12)
plt.grid(True, alpha=0.3)

# Ajout d'une ligne de tendance
z = np.polyfit(df['Temp'], df['GLOBAL'], 1)
p = np.poly1d(z)
plt.plot(df['Temp'].sort_values(), p(df['Temp'].sort_values()), 
         "r--", linewidth=2, label=f'Tendance: y={z[0]:.2f}x+{z[1]:.2f}')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visualisation 5: Évolution mensuelle avec comparaison par année
df['Mois'] = df['Date'].dt.month
df['Mois_Nom'] = df['Date'].dt.strftime('%b')

presence_mensuelle = df.groupby(['Annee', 'Mois'])['GLOBAL'].mean().reset_index()

plt.figure(figsize=(14, 6))
for annee in df['Annee'].unique():
    data_annee = presence_mensuelle[presence_mensuelle['Annee'] == annee]
    plt.plot(data_annee['Mois'], data_annee['GLOBAL'], marker='o', 
             linewidth=2, label=f'Année {int(annee)}')

plt.title('Taux de présence moyen mensuel par année', fontsize=14, fontweight='bold')
plt.xlabel('Mois', fontsize=12)
plt.ylabel('Taux de présence moyen', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(range(1, 13), ['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Juin', 
                           'Juil', 'Août', 'Sep', 'Oct', 'Nov', 'Déc'])
plt.tight_layout()
plt.show()

In [ ]:
# Visualisation 6: Heatmap - Taux de présence par jour de semaine et semaine de l'année
pivot_data = df.pivot_table(
    values='GLOBAL', 
    index='jour_semaine', 
    columns='Semaine', 
    aggfunc='mean'
)

# Réordonner les jours de la semaine
jour_ordre = ['lundi', 'mardi', 'mercredi', 'jeudi', 'vendredi']
pivot_data = pivot_data.reindex(jour_ordre)

plt.figure(figsize=(20, 6))
plt.imshow(pivot_data, cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Taux de présence moyen')
plt.title('Heatmap: Taux de présence par jour de la semaine et semaine de l\'année', 
          fontsize=14, fontweight='bold')
plt.xlabel('Semaine de l\'année', fontsize=12)
plt.ylabel('Jour de la semaine', fontsize=12)
plt.yticks(range(len(jour_ordre)), jour_ordre)
plt.tight_layout()
plt.show()

In [ ]:
# Visualisation 7: Impact des jours fériés et ponts
fig, axes = plt.subplots(1, 5, figsize=(15, 5))

# Jours fériés
jour_ferie_stats = df.groupby('jour_ferie.')['GLOBAL'].mean()
axes[0].bar(['Jour normal', 'Jour férié'], jour_ferie_stats.values, 
            color=['#457B9D', '#E63946'])
axes[0].set_title('Impact des jours fériés', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Taux de présence moyen', fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')

# Ponts/Congés
pont_stats = df.groupby('pont.conge.')['GLOBAL'].mean()
axes[1].bar(['Jour normal', 'Pont/Congé'], pont_stats.values, 
            color=['#457B9D', '#F4A261'])
axes[1].set_title('Impact des ponts/congés', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Taux de présence moyen', fontsize=10)
axes[1].grid(True, alpha=0.3, axis='y')

# Grèves
greve_stats = df.groupby('Greve_nationale')['GLOBAL'].mean()
axes[2].bar(['Jour normal', 'Greve_nationale'], greve_stats.values, 
            color=['#457B9D', '#E76F51'])
axes[2].set_title('Impact des grèves', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Taux de présence moyen', fontsize=10)
axes[2].grid(True, alpha=0.3, axis='y')

# prof_nationale
prof_stats = df.groupby('prof_nationale')['GLOBAL'].mean()
axes[3].bar(['Jour normal', 'prof_nationale'], prof_stats.values, 
            color=['#457B9D', '#6A994E'])
axes[3].set_title('Impact des profs nationaux', fontsize=12, fontweight='bold')
axes[3].set_ylabel('Taux de présence moyen', fontsize=10)
axes[3].grid(True, alpha=0.3, axis='y')

# Autres
autre_stats = df.groupby('autre')['GLOBAL'].mean()
axes[4].bar(['Jour normal', 'Autre'], autre_stats.values, 
            color=['#457B9D', '#C73E1D'])
axes[4].set_title('Impact des autres événements', fontsize=12, fontweight='bold')
axes[4].set_ylabel('Taux de présence moyen', fontsize=10)
axes[4].grid(True, alpha=0.3, axis='y')

# Ajout des valeurs
for ax in axes:
    for bar in ax.patches:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.0f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Visualisation 8: Statistiques descriptives des réservations
stats = df[['GLOBAL', 'D1', 'D2', 'D3', 'D4']].describe()

fig, ax = plt.subplots(figsize=(12, 6))
bp = ax.boxplot([df['GLOBAL'], df['D1'], df['D2'], df['D3'], df['D4']], 
                 labels=['GLOBAL', 'D1', 'D2', 'D3', 'D4'],
                 patch_artist=True)

# Coloration des boîtes
colors = ['#1D3557', '#A23B72', '#F18F01', '#C73E1D', '#6A994E']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_title('Distribution du taux de présence par catégorie', fontsize=14, fontweight='bold')
ax.set_ylabel('Taux de présence', fontsize=12)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\nStatistiques descriptives:")
print(stats)